

# Access DB2 on Cloud using Python



## Objectives

After completing this lab you will be able to:

*   Create a table
*   Insert data into the table
*   Query data from the table
*   Retrieve the result set into a pandas dataframe
*   Close the database connection


**Notice:** Please follow the instructions given in the first Lab of this course to Create a database service instance of Db2 on Cloud.

## Task 1: Import the `ibm_db` Python library

The `ibm_db` [API ](https://pypi.python.org/pypi/ibm_db/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01) provides a variety of useful Python functions for accessing and manipulating data in an IBM® data server database, including functions for connecting to a database, preparing and issuing SQL statements, fetching rows from result sets, calling stored procedures, committing and rolling back transactions, handling errors, and retrieving metadata.

We import the ibm_db library into our Python Application

The following required modules are pre-installed in the Skills Network Labs environment. However if you run this notebook commands in a different Jupyter environment (e.g. Watson Studio or Ananconda) you may need to install these libraries by removing the `#` sign before `!pip` in the code cell below.


In [14]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
!pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
!pip install ipython-sql

  Using cached ibm_db-3.1.0-py3-none-any.whl
  Using cached ibm_db_sa-0.3.3-py3-none-any.whl
     ---------------------------------------- 2.1/2.1 MB 3.4 MB/s eta 0:00:00
  Using cached greenlet-3.0.3-cp39-cp39-win_amd64.whl (290 kB)
  Attempting uninstall: ibm_db
    Found existing installation: ibm-db 3.1.0
    Uninstalling ibm-db-3.1.0:


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\nh\\anaconda3\\lib\\site-packages\\clidriver\\bin\\amd64.VC12.CRT\\msvcr120.dll'
Consider using the `--user` option or check the permissions.



Found existing installation: SQLAlchemy 1.3.24
Uninstalling SQLAlchemy-1.3.24:
  Successfully uninstalled SQLAlchemy-1.3.24
  Using cached SQLAlchemy-1.3.24-cp39-cp39-win_amd64.whl (1.2 MB)


In [1]:
import ibm_db

When the command above completes, the `ibm_db` library is loaded in your notebook.

## Task 2: Identify the database connection credentials

Connecting to dashDB or DB2 database requires the following information:

*   Driver Name
*   Database name
*   Host DNS name or IP address
*   Host port
*   Connection protocol
*   User ID
*   User Password

**Notice:** To obtain credentials please refer to the instructions given in the first Lab of this course

Now enter your database credentials below

Replace the placeholder values in angular brackets <> below with your actual database credentials

e.g. replace "database" with "BLUDB"


In [4]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud" # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "vvc90062"        # e.g. "abc12345"
dsn_pwd = "j2qPN1zj4vXhHFYu"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "bludb"            # e.g. "BLUDB"
dsn_port =  "31929"               # e.g. "32733" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"

## Task 3: Create the database connection

Ibm_db API uses the IBM Data Server Driver for ODBC and CLI APIs to connect to IBM DB2 and Informix.

Create the database connection


In [5]:
#Create database connection
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
dsn = ("DRIVER={0};DATABASE={1};HOSTNAME={2};PORT={3};PROTOCOL={4};UID={5};PWD={6};SECURITY={7};").format(dsn_driver,
                                                                                                     dsn_database, 
                                                                                                          dsn_hostname,
                                                                                                          dsn_port,
                                                                                                          dsn_protocol,
                                                                                                          dsn_uid,
                                                                                                          dsn_pwd,
                                                                                                          dsn_security)

print(dsn)
conn = ibm_db.connect(dsn, "", "")
print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)



DRIVER={IBM DB2 ODBC DRIVER};DATABASE=bludb;HOSTNAME=55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud;PORT=31929;PROTOCOL=TCPIP;UID=vvc90062;PWD=j2qPN1zj4vXhHFYu;SECURITY=SSL;
Connected to database:  bludb as user:  vvc90062 on host:  55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud


## Task 4: Create a table in the database

In this step we will create a table in the database with following details:

<img src = "https://ibm.box.com/shared/static/ztd2cn4xkdoj5erlk4hhng39kbp63s1h.jpg" align="center">


In [6]:
#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt
dropQuery = "drop table INSTRUCTOR"

#Now execute the drop statment
dropStmt = ibm_db.exec_immediate(conn, dropQuery)
#dropStmt = ibm_db.exec_immediate(conn, "drop table INSTRUCTOR")

Exception: [IBM][CLI Driver][DB2/LINUXX8664] SQL0204N  "VVC90062.INSTRUCTOR" is an undefined name.  SQLSTATE=42704 SQLCODE=-204

## Dont worry if you get this error:

If you see an exception/error similar to the following, indicating that INSTRUCTOR is an undefined name, that's okay. It just implies that the INSTRUCTOR table does not exist in the table - which would be the case if you had not created it previously.

Exception: \[IBM]\[CLI Driver]\[DB2/LINUXX8664] SQL0204N  "ABC12345.INSTRUCTOR" is an undefined name.  SQLSTATE=42704 SQLCODE=-204


In [7]:
#Construct the Create Table DDL statement - replace the ... with rest of the statement
createQuery = '''create table INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL,
                                         FNAME VARCHAR(20), LNAME VARCHAR(20),
                                         CITY VARCHAR(20), CCODE CHAR(2))'''

#Now fill in the name of the method and execute the statement
createStmt = ibm_db.exec_immediate(conn, createQuery)

In [8]:
createStmt

<details><summary>Click here for the solution</summary>

```python
createQuery = "create table INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2))"

createStmt = ibm_db.exec_immediate(conn,createQuery)
```

</details>


## Task 5: Insert data into the table

In this step we will insert some rows of data into the table.

The INSTRUCTOR table we created in the previous step contains 3 rows of data:

<img src="https://ibm.box.com/shared/static/j5yjassxefrjknivfpekj7698dqe4d8i.jpg" align="center">

We will start by inserting just the first row of data, i.e. for instructor Rav Ahuja


In [9]:
#Construct the query - replace ... with the insert statement
insertQuery = "insert into INSTRUCTOR values (1, 'Rav', 'Ahuja', 'TORONTO', 'CA')"

#execute the insert statement
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

Now use a single query to insert the remaining two rows of data


In [10]:
#replace ... with the insert statement that inerts the remaining two rows of data
insertQuery2 = "insert into INSTRUCTOR values (2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')"

#execute the statement
insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)

## Task 6: Query data in the table

In this step we will retrieve data we inserted into the INSTRUCTOR table.


In [11]:
#Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Fetch the Dictionary (for the first row only) - replace ... with your code
row1=ibm_db.fetch_both(selectStmt)
row1

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

In [12]:
ibm_db.result(selectStmt,"ID")

1

In [13]:
ibm_db.result(selectStmt,0)

1

In [14]:
ibm_db.result(selectStmt,'FNAME')

'Rav'

In [16]:
row1["ID"]

1

In [17]:
row1[1]

'Rav'

In [18]:
row1["FNAME"]

'Rav'

In [19]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
   print (" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

 ID: 2  FNAME: Raul
 ID: 3  FNAME: Hima


In [20]:
#Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)



In [21]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
   print (" ID:",  ibm_db.result(selectStmt, "ID"), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

 ID: 1  FNAME: Rav
 ID: 2  FNAME: Raul
 ID: 3  FNAME: Hima


Bonus: now write and execute an update statement that changes the Rav's CITY to MOOSETOWN


In [22]:
#Enter your code below
updateQuery = "update INSTRUCTOR set CITY='MOOSETOWN' where FNAME='Rav'"
updateStmt = ibm_db.exec_immediate(conn, updateQuery)

In [23]:
#Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)



In [24]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
   print (" FNAME:",  ibm_db.result(selectStmt, "FNAME")," CITY:",  ibm_db.result(selectStmt, "CITY"))

 FNAME: Rav  CITY: MOOSETOWN
 FNAME: Raul  CITY: Markham
 FNAME: Hima  CITY: Chicago


## Task 7: Retrieve data into Pandas

In this step we will retrieve the contents of the INSTRUCTOR table into a Pandas dataframe


In [25]:
import pandas as pd
import ibm_db_dbi

In [26]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)#pcon is the connection with pandas

In [27]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#retrieve the query results into a pandas dataframe
pdf = pd.read_sql(selectQuery, pconn)

#print data frame
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [28]:
pdf.LNAME[0]

'Ahuja'

In [29]:
pdf["LNAME"][0]

'Ahuja'

Once the data is in a Pandas dataframe, you can do the typical pandas operations on it.

For example you can use the shape method to see how many rows and columns are in the dataframe


In [30]:
pdf.shape

(3, 5)

In [31]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select id,fname from INSTRUCTOR"

#retrieve the query results into a pandas dataframe
pdf2 = pd.read_sql(selectQuery, pconn)
pdf2

,ID,FNAME
0,1,Rav
1,2,Raul
2,3,Hima


In [32]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR where LCASE(CCODE)='ca'"

#retrieve the query results into a pandas dataframe
pdf3 = pd.read_sql(selectQuery, pconn)
pdf3

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA


In [33]:
pdf3.loc[0,"FNAME"]="sheirf"

In [34]:
pdf3

,ID,FNAME,LNAME,CITY,CCODE
0,1,sheirf,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA


In [35]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR where LOWER(CCODE)='ca'"

#retrieve the query results into a pandas dataframe
pdf3 = pd.read_sql(selectQuery, pconn)
pdf3

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA


## Task 8: Close the Connection

We free all resources by closing the connection. Remember that it is always important to close connections so that we can avoid unused connections taking up resources.


In [36]:
ibm_db.close(conn)

True